In [6]:
from collections import deque
import copy
import time
import random
import statistics
from dataclasses import dataclass, field

In [7]:
@dataclass
class Board:
    board: list = field(default_factory=lambda: [["*"] * 9 for _ in range(9)])

    def __init__(self, board=None):
        if board is None:
            self.board = [["*"] * 9 for _ in range(9)]
        else:
            self.board = board

    def show(self):
        self.print_board(self.board)

    def print_board(self, board):
        self.print_upper_labels()
        for line_number, line in enumerate(board):
            self.print_line_separators(line_number)
            self.print_columns(line, line_number)
            print()

    def print_upper_labels(self):
        print('  A  B  C    D  E  F    G  H  I')

    def print_line_separators(self, line_number):
        if line_number % 3 == 0 and line_number != 0:
            print(" ", "-" * 30)

    def print_columns(self, line, line_number):
        print(f"{line_number+1} ", end="")
        for index, place in enumerate(line):
            if index % 3 == 0 and index != 0:
                print("|", end=" ")
            print(place, end="  ")

    def update(self, new_board):
        self.board = new_board

In [8]:
def is_valid(board, row, col, num):                    ## verifica se um numero pode ser inserido em uma posicao especifica no tabuleiro 
    for i in range(9):
        if board[row][i] == num:                       ## se tiver um numero igual na linha = NAO PODE
            return False
    for i in range(9):
        if board[i][col] == num:                       ## se tiver um numero igual na coluna = NAO PODE
            return False
    start_row = (row // 3) * 3
    start_col = (col // 3) * 3
    for i in range(3):                                 ## considera a subgrade 3x3, se tiver igual = NAO PODE
        for j in range(3):
            if board[start_row + i][start_col + j] == num:
                return False
    return True

In [9]:
def find_empty(board): ### busca 0, onde 0 significa SEM NUMERO - retorna (linha,coluna)
    for i in range(9):
        for j in range(9):
            if board[i][j] == 0:
                return (i, j)
    return None

In [10]:
def bfs_sudoku_solver(initial_board):                     # initial board = o que o usuario definiu
    queue = [initial_board]                               # transforma o primeiro estado em uma lista
    
    while queue:                                          # loop para rodar em todos os elementos do initial_board - uma lista
        current_board = queue.pop(0)                      # remove o primeiro elemento da lista e adiciona-o em current_board - o novo estado do board
        empty_position = find_empty(current_board)        # busca zeros, pega suas coordenadas
        
        if not empty_position:                            # SE nao tiver zeros, fim - retorna o current_board
            return current_board  
        
        row, col = empty_position                         # coordenadas empty position
        
        for num in range(1, 10):                          # inicia testes para os numeros de 1 a 9, para tentar alocar um que seja a solucao
            if is_valid(current_board, row, col, num):    # puxa a funcao que verifica se o numero teste eh valido ou nao
                new_board = copy.deepcopy(current_board)  # deep copy para separar totalmente os boards
                new_board[row][col] = num                 # insere o num como valido na posicao linha,coluna achado anteriormente
                queue.append(new_board)                   # adiciona o novo estado do tabuleiro no final da lista 
    
    return None                                           # Nenhuma solução encontrada

In [11]:
def dfs_sudoku_solver(initial_board):                     
    stack = [initial_board]                               # transforma o primeiro estado em uma pilha (na pratica igual a lista)
    
    while stack:                                          
        current_board = stack.pop()                       # remove o ULTIMO elemento da lista e adiciona-o em current_board - o novo estado do board
        empty_position = find_empty(current_board)        # busca zeros, pega suas coordenadas
        
        if not empty_position:                            
            return current_board  
        
        row, col = empty_position                         # coordenadas empty position
        
        for num in range(1, 10):                          # inicia testes para os números de 1 a 9, para tentar alocar um que seja a solução
            if is_valid(current_board, row, col, num):    # puxa a função que verifica se o número teste é válido ou não
                new_board = copy.deepcopy(current_board)  # deep copy para separar totalmente os boards
                new_board[row][col] = num                 # insere o num como válido na posição linha, coluna achada anteriormente
                stack.append(new_board)                   # adiciona o novo estado do tabuleiro no TOPO da pilha
    
    return None                                           


In [12]:
def input_sudoku():
    board = []
    print("Digite o estado inicial do Sudoku (use 0 para células vazias):")
    for i in range(9):
        row = list(map(int, input(f"Digite a linha {i+1} (9 números separados por espaço): ").split()))
        board.append(row)
    return board

In [28]:
def generate_random_sudoku():
    board = [[0 for _ in range(9)] for _ in range(9)]
    for _ in range(random.randint(10, 20)):  
        row, col = random.randint(0, 8), random.randint(0, 8)
        num = random.randint(1, 9)
        if is_valid(board, row, col, num):
            board[row][col] = num
    return board

In [29]:
def test_algorithms(num_tests=10):
    bfs_times = []
    dfs_times = []
    for _ in range(num_tests):
        board = generate_random_sudoku()
        # Testa BFS
        start_time = time.time()
        bfs_sudoku_solver(board)
        took_bfs = time.time() - start_time
        print(took_bfs)
        print('BFS DONE')
        bfs_times.append(took_bfs)
        # Testa DFS
        start_time = time.time()
        dfs_sudoku_solver(board)
        took_dfs = time.time() - start_time
        print(took_dfs)
        print('DFS DONE')
        dfs_times.append(took_dfs)
    bfs_mean = statistics.mean(bfs_times)
    bfs_stdev = statistics.stdev(bfs_times)
    dfs_mean = statistics.mean(dfs_times)
    dfs_stdev = statistics.stdev(dfs_times)

    print(f"BFS - Tempo Médio: {bfs_mean:.4f} s, Desvio Padrão: {bfs_stdev:.4f} s")
    print(f"DFS - Tempo Médio: {dfs_mean:.4f} s, Desvio Padrão: {dfs_stdev:.4f} s")

In [27]:
if __name__ == "__main__":
    initial_board = input_sudoku()
    board_instance = Board(board=initial_board)
    board_instance.show()

    print("\nResolvendo com Busca em Largura (BFS)...")
    solution = bfs_sudoku_solver(initial_board)
    if solution:
        board_instance.update(solution)
        board_instance.show()
    else:
        print("Nenhuma solução encontrada com BFS.")

    print("\nResolvendo com Busca em Profundidade (DFS)...")
    solution = dfs_sudoku_solver(initial_board)
    if solution:
        board_instance.update(solution)
        board_instance.show()
    else:
        print("Nenhuma solução encontrada com DFS.")
    

    print("\nTestando o desempenho dos algoritmos...")
    #test_algorithms()

Digite o estado inicial do Sudoku (use 0 para células vazias):


Digite a linha 1 (9 números separados por espaço):  1 0 7 0 0 6 4 5 0
Digite a linha 2 (9 números separados por espaço):  0 2 5 3 4 0 0 0 8
Digite a linha 3 (9 números separados por espaço):  0 6 0 0 0 1 0 7 0
Digite a linha 4 (9 números separados por espaço):  0 5 3 0 0 0 0 2 9
Digite a linha 5 (9 números separados por espaço):  6 1 0 0 0 9 8 0 0
Digite a linha 6 (9 números separados por espaço):  0 0 0 6 0 2 0 0 7
Digite a linha 7 (9 números separados por espaço):  0 0 1 0 9 3 2 0 0
Digite a linha 8 (9 números separados por espaço):  0 0 8 0 0 0 0 0 0
Digite a linha 9 (9 números separados por espaço):  0 4 0 0 7 8 5 9 1 


  A  B  C    D  E  F    G  H  I
1 1  0  7  | 0  0  6  | 4  5  0  
2 0  2  5  | 3  4  0  | 0  0  8  
3 0  6  0  | 0  0  1  | 0  7  0  
  ------------------------------
4 0  5  3  | 0  0  0  | 0  2  9  
5 6  1  0  | 0  0  9  | 8  0  0  
6 0  0  0  | 6  0  2  | 0  0  7  
  ------------------------------
7 0  0  1  | 0  9  3  | 2  0  0  
8 0  0  8  | 0  0  0  | 0  0  0  
9 0  4  0  | 0  7  8  | 5  9  1  

Resolvendo com Busca em Largura (BFS)...
  A  B  C    D  E  F    G  H  I
1 1  3  7  | 9  8  6  | 4  5  2  
2 9  2  5  | 3  4  7  | 1  6  8  
3 8  6  4  | 5  2  1  | 9  7  3  
  ------------------------------
4 7  5  3  | 8  1  4  | 6  2  9  
5 6  1  2  | 7  3  9  | 8  4  5  
6 4  8  9  | 6  5  2  | 3  1  7  
  ------------------------------
7 5  7  1  | 4  9  3  | 2  8  6  
8 2  9  8  | 1  6  5  | 7  3  4  
9 3  4  6  | 2  7  8  | 5  9  1  

Resolvendo com Busca em Profundidade (DFS)...
  A  B  C    D  E  F    G  H  I
1 1  3  7  | 9  8  6  | 4  5  2  
2 9  2  5  | 3  4  7  | 1  6  8  
3 8

In [ ]:
""""
SUDOKU TESTE
Digite o estado inicial do Sudoku (use 0 para células vazias):
Digite a linha 1 (9 números separados por espaço):  1 0 7 0 0 6 4 5 0
Digite a linha 2 (9 números separados por espaço):  0 2 5 3 4 0 0 0 8
Digite a linha 3 (9 números separados por espaço):  0 6 0 0 0 1 0 7 0
Digite a linha 4 (9 números separados por espaço):  0 5 3 0 0 0 0 2 9
Digite a linha 5 (9 números separados por espaço):  6 1 0 0 0 9 8 0 0
Digite a linha 6 (9 números separados por espaço):  0 0 0 6 0 2 0 0 7
Digite a linha 7 (9 números separados por espaço):  0 0 1 0 9 3 2 0 0
Digite a linha 8 (9 números separados por espaço):  0 0 8 0 0 0 0 0 0
Digite a linha 9 (9 números separados por espaço):  0 4 0 0 7 8 5 9 1 
"""""